In [ ]:
# from netCDF4 import Dataset
import numpy as np
import xarray as xr
import tensorflow as tf
# from tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Dense, Flatten, Conv3DTranspose, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout

In [ ]:
# Set NumPy to display the full array
# np.set_printoptions(threshold=np.inf)  # Show entire array, no truncation

In [ ]:
# https://drive.google.com/file/d/1od-TIaU_oFTKZZEXsvOglM5Qqz3c6CV8/view?usp=drive_link
# https://drive.google.com/file/d/113t5V1yFetinTzClFCsIJU3JqdM9uIeh/view?usp=sharing, 
# https://drive.google.com/file/d/1MT26XfWRrGYFq4OmYvlVa8tvYm_vWRs5/view?usp=sharing, 
# https://drive.google.com/file/d/1ZAt95Bac_GNrsI3-eKg6QJIlVK6CSwgM/view?usp=sharing, 
# https://drive.google.com/file/d/1t7g4sm_Ri_99t9okGi0tgcAJvUQ7YTTk/view?usp=sharing, 
# https://drive.google.com/file/d/1u1O2ccFdpXdMGMVIA0Ew2mPHOnoTfYLg/view?usp=sharing
url = 'https://drive.google.com/uc?id=1od-TIaU_oFTKZZEXsvOglM5Qqz3c6CV8'

# Download the file
!gdown {url}
url = 'https://drive.google.com/uc?id=113t5V1yFetinTzClFCsIJU3JqdM9uIeh'

# Download the file
!gdown {url}
url = 'https://drive.google.com/uc?id=1MT26XfWRrGYFq4OmYvlVa8tvYm_vWRs5'

# Download the file
!gdown {url}
url = 'https://drive.google.com/uc?id=1ZAt95Bac_GNrsI3-eKg6QJIlVK6CSwgM'

# Download the file
!gdown {url}
url = 'https://drive.google.com/uc?id=1t7g4sm_Ri_99t9okGi0tgcAJvUQ7YTTk'

# Download the file
!gdown {url}
url = 'https://drive.google.com/uc?id=1u1O2ccFdpXdMGMVIA0Ew2mPHOnoTfYLg'

# Download the file
!gdown {url}

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Example: Simulated dataset
# num_samples = 6  # Number of samples
# temporal_depth = 2  # Number of consecutive years as input
# height, width, num_bands = 2240, 2190, 7  # Image dimensions and bands
# num_classes = 5  # Number of discrete classes

In [ ]:
import xarray as xr
import numpy as np
import os

# List of NetCDF file paths
file_paths = ['/kaggle/working/landsat2000.nc', '/kaggle/working/landsat2005.nc',
              '/kaggle/working/landsat2010.nc', '/kaggle/working/landsat2015_2.nc',
              '/kaggle/working/landsat2020_2.nc', '/kaggle/working/landsat2024_2.nc']  # Replace with your file paths

# Name of the variable you want to extract
# variable_name = 'band1'  # Replace with the actual variable name

# Initialize an empty list to store NumPy arrays
data_list = []

# Loop through each file, load the variable, and append to the list
for file_path in file_paths:
    temporarial = []
    if os.path.exists(file_path):  # Check if the file exists
        print(f"Loading file: {file_path}")

        # Open the file using xarray
        dataset = xr.open_dataset(file_path)
        dataset = dataset.fillna(0)
        for variable_name in ['band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7']:
            # Extract the variable as a NumPy array and append to the list
            data_array = dataset[variable_name].values  # Extract data as NumPy array
            temporarial.append(data_array)
            # print(f"Shape of data from {file_path}: {data_array.shape}")
    data_list.append(temporarial)



# At this point, 'combined_data' contains all the concatenated data.


In [ ]:
np.array(data_list).shape

In [ ]:
final_array = np.array(data_list)

In [ ]:
# Reorder axes to (samples, height, width, number_of_channels)
array_reordered = np.transpose(final_array, (0, 2, 3, 1))

In [ ]:
array_reordered.shape

In [ ]:
def normalize_with_padding_dataset(images):
    """
    Normalizes a dataset of images to the range [0, 1] channel-wise across all samples,
    leaving zero-padding unchanged.

    Args:
        images (numpy.ndarray): Dataset of images with shape (N, H, W, C), where
                                N is the number of samples, H and W are spatial dimensions,
                                and C is the number of channels.

    Returns:
        numpy.ndarray: Normalized dataset with padding (zeros) unchanged.
        list: List of (min, max) values for each channel across the entire dataset.
    """
    # Check the shape of the input array
    if images.ndim != 4:
        raise ValueError("Input dataset must be a 4D array with shape (N, H, W, C).")
    
    # Initialize min-max tracker for each channel
    min_max_values = []
    normalized_images = images.copy()
    
    # Normalize channel-wise across all samples
    for c in range(images.shape[-1]):
        # Extract all channel values across all images
        channel_values = images[:, :, :, c]
        non_zero_mask = channel_values != 0
        
        # Compute min and max across all non-zero values for the channel
        arr_min = channel_values[non_zero_mask].min() if non_zero_mask.any() else 0
        arr_max = channel_values[non_zero_mask].max() if non_zero_mask.any() else 1
        
        # Normalize the channel values for all images
        if arr_max > arr_min:  # Avoid division by zero
            normalized_images[:, :, :, c][non_zero_mask] = (
                channel_values[non_zero_mask] - arr_min
            ) / (arr_max - arr_min)
        
        # Save min-max values for the channel
        min_max_values.append((arr_min, arr_max))
    
    return normalized_images, min_max_values

In [ ]:
array_reordered,min_max_values = normalize_with_padding_dataset(array_reordered)

In [ ]:
array_reordered.max()

In [ ]:
x = np.arange(array_reordered[0].shape[1])  # Height indices
y = np.arange(array_reordered[0].shape[0])  # Width indices

In [ ]:
# Band names
band_names = [f"band{i+1}" for i in range(7)]

# Create a dictionary to store each band as a separate variable
data_vars = {
    band_names[i]: (["y", "x"], array_reordered[0][:, :, i]) for i in range(7)
}

In [ ]:
# Create an xarray Dataset
ds = xr.Dataset(
    {
        band_names[i]: (["y", "x"], array_reordered[0][:, :, i]) for i in range(7)
    },
    coords={
        "x": x,  # Spatial x-coordinates
        "y": y,  # Spatial y-coordinates
    }
)

In [ ]:
x = ds['x']
y = ds['y']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bands = ["band1", "band2", "band3", "band4", "band5", "band6", "band7"]  # Replace with actual names

fig, axes = plt.subplots(3, 3, figsize=(15, 15))  # Adjust grid size as needed
axes = axes.flatten()

for i, band in enumerate(bands):
    if i < len(axes):
        data_band = ds[band]
        axes[i].pcolormesh(x, y, data_band, shading='auto', cmap='viridis')
        axes[i].set_title(band)
        axes[i].set_xlabel("X Coordinate")
        axes[i].set_ylabel("Y Coordinate")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

def preprocess_netcdf_data(data, num_frames=2):
    """
    Prepares data for training a Conv3D model with temporal input sequences.

    Args:
        data (numpy.ndarray): The input data of shape (num_timesteps, height, width, channels).
        num_frames (int): Number of input frames for each sequence.

    Returns:
        X (numpy.ndarray): Input sequences of shape (num_samples, num_frames, height, width, channels).
        y (numpy.ndarray): Target frames of shape (num_samples, height, width, channels).
    """
    # Number of time steps in the data
    num_timesteps, height, width, channels = data.shape

    # Create input (X) and target (y) sequences
    X = []
    y = []

    for i in range(num_timesteps - num_frames):
        X.append(data[i:i + num_frames])  # Take 'num_frames' consecutive frames as input
        y.append(data[i + num_frames])   # The next frame as target

    # Convert to NumPy arrays
    X = np.array(X)  # Shape: (num_samples, num_frames, height, width, channels)
    y = np.array(y)  # Shape: (num_samples, height, width, channels)

    return X, y

In [ ]:
X, y = preprocess_netcdf_data(array_reordered[:6], num_frames=1)

In [ ]:
X = np.squeeze(X)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
def extract_kernels_to_numpy(images, kernel_size=3, output_file="kernel_data.npy"):
    """
    Extract 3x3 kernels and store them as a NumPy array of shape (n, 9*channels).

    Parameters:
        images (numpy.ndarray): Input array of shape (samples, height, width, channels).
        kernel_size (int): Size of the kernel (default is 3).
        output_file (str): Filepath to save the output NumPy array.

    Returns:
        None: Saves the result as a .npy file.
    """
    pad = kernel_size // 2

    # Padding the images to handle border pixels
    padded_images = np.pad(images, ((0, 0), (pad, pad), (pad, pad), (0, 0)), mode='constant')

    # Get the total number of kernels for preallocation
    num_images, height, width, channels = images.shape
    num_kernels = num_images * height * width

    # Preallocate a large NumPy matrix for storage
    result_matrix = np.zeros((num_kernels, kernel_size * kernel_size * channels), dtype=images.dtype)

    # Fill the matrix efficiently
    row = 0
    for img_index in range(padded_images.shape[0]):
        image = padded_images[img_index]
        for i in range(pad, image.shape[0] - pad):
            for j in range(pad, image.shape[1] - pad):
                # Extract the 3x3 kernel for all channels and flatten
                kernel = image[i-pad:i+pad+1, j-pad:j+pad+1, :].flatten()
                # Assign to the result matrix
                result_matrix[row, :] = kernel
                row += 1

    return result_matrix


In [ ]:
X_train = extract_kernels_to_numpy(X, kernel_size=3)

In [ ]:
x_train, X_test = X_train[:19622400],X_train[19622400:]

In [ ]:
def extract_pixels_to_numpy(images):
    """
    Extract individual pixels and their channel values, storing them as a NumPy array of shape (n, channels).

    Parameters:
        images (numpy.ndarray): Input array of shape (samples, height, width, channels).
        output_file (str): Filepath to save the output NumPy array.

    Returns:
        None: Saves the result as a .npy file.
    """
    # Get dimensions of the input images
    num_images, height, width, channels = images.shape
    num_pixels = num_images * height * width

    # Preallocate a large NumPy matrix for storage
    result_matrix = np.zeros((num_pixels, channels), dtype=images.dtype)

    # Fill the matrix efficiently
    row = 0
    for img_index in range(num_images):
        image = images[img_index]
        for i in range(height):
            for j in range(width):
                # Extract the pixel values for all channels
                pixel = image[i, j, :]
                # Assign to the result matrix
                result_matrix[row, :] = pixel
                row += 1
    return result_matrix


In [ ]:
Y_data = extract_pixels_to_numpy(y)

In [ ]:
y_train, y_test = Y_data[:19622400],Y_data[19622400:]

In [ ]:
x_train.shape,y_train.shape

In [ ]:
# Step 1: Identify zero rows
zero_row_indices_x_train = np.where(~x_train.any(axis=1))[0]

# Step 2: Remove zero rows
non_zero_array_x_train = x_train[x_train.any(axis=1)]

In [ ]:
# Remove rows at zero_row_indices from array2
non_zero_array_y_train = np.delete(y_train, zero_row_indices_x_train, axis=0)

In [ ]:
non_zero_array_x_train.shape,non_zero_array_y_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
non_zero_array_x_train = np.expand_dims(non_zero_array_x_train,axis=1)

In [ ]:
X_test = np.expand_dims(X_test,axis=1)

In [ ]:
non_zero_array_x_train.shape

In [ ]:
# Define the input shape
# input_layer = Input(shape=(1, non_zero_array_x_train.shape[2]))

# # LSTM layer with return_sequences=True to capture hidden states
# lstm_layer = LSTM(50, activation='relu', return_sequences=True, return_state=True)

# # Pass input through LSTM
# lstm_output, hidden_state, cell_state = lstm_layer(input_layer)

# # Add a Dense layer for the final LSTM predictions
# dense_layer = Dense(7)(hidden_state)

# # Define the model
# model = Model(inputs=input_layer, outputs=dense_layer)

# Compile the model
# lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
num_samples = 1000
timesteps = 10
features = 5


# Define the model
model = Sequential([
    LSTM(32, input_shape=(1, non_zero_array_x_train.shape[2])),  # LSTM layer with 32 units
    Dense(16, activation='relu'),                # Dense layer with 16 units
    Dense(7, activation='sigmoid')               # Output layer for binary classification
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='huber', metrics=['mae'])

# Train the model on GPU (this will happen automatically if GPU is enabled in Colab)
model.fit(non_zero_array_x_train, non_zero_array_y_train, epochs=5, batch_size=1024, validation_data=(X_test, y_test))

In [ ]:
def reconstruct_image_from_numpy(pixel_data, original_shape):
    """
    Reconstruct the original image from a flattened pixel data array.

    Parameters:
        pixel_data (numpy.ndarray): Input array of shape (n, channels).
        original_shape (tuple): The original shape of the image array (samples, height, width, channels).

    Returns:
        numpy.ndarray: Reconstructed image array with the original shape.
    """
    # Ensure the total number of pixels matches
    expected_pixels = original_shape[0] * original_shape[1] * original_shape[2]
    if pixel_data.shape[0] != expected_pixels:
        raise ValueError("The number of pixels in the pixel data does not match the original image shape.")

    # Reshape the pixel data to the original image shape
    reconstructed_image = pixel_data.reshape(original_shape)
    return reconstructed_image

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Define the original shape of the images (samples, height, width, channels)
original_shape = (1,2190, 2240, 7)

# Reconstruct the image
reconstructed_images = reconstruct_image_from_numpy(y_pred, original_shape)

In [ ]:
pred_original_shape = np.squeeze(reconstructed_images)

In [ ]:
pred_original_shape.shape

In [ ]:
def denormalize_dataset(normalized_images, min_max_values):
    """
    Denormalizes a dataset of images from the range [0, 1] channel-wise to the original scale.

    Args:
        normalized_images (numpy.ndarray): Normalized dataset with shape (N, H, W, C).
        min_max_values (list): List of (min, max) tuples for each channel.

    Returns:
        numpy.ndarray: Denormalized dataset with the original scale restored.
    """
    # Check the shape of the input array
    if normalized_images.ndim != 4:
        raise ValueError("Input dataset must be a 4D array with shape (N, H, W, C).")
    
    if len(min_max_values) != normalized_images.shape[-1]:
        raise ValueError("Length of min_max_values must match the number of channels in the dataset.")
    
    # Initialize the output array
    denormalized_images = normalized_images.copy()
    
    # Denormalize each channel across all images
    for c in range(normalized_images.shape[-1]):
        channel_values = normalized_images[:, :, :, c]
        arr_min, arr_max = min_max_values[c]
        
        # Reverse the normalization
        if arr_max > arr_min:  # Avoid division by zero
            denormalized_images[:, :, :, c] = channel_values * (arr_max - arr_min) + arr_min
    
    return denormalized_images

In [ ]:
# Rescale normalized arrays back to their original values
y_rescaled = denormalize_dataset(reconstructed_images, min_max_values)

In [ ]:
y_rescaled.shape

In [ ]:
y_rescaled = np.squeeze(y_rescaled)

In [ ]:
y_rescaled.shape

In [ ]:
# Step 3: Reinsert zero rows
# def reinsert_zero_rows(array, zero_indices, original_shape):
#     result = np.zeros(original_shape, dtype=array.dtype)
#     non_zero_row_idx = 0
    
#     for i in range(original_shape[0]):
#         if i in zero_indices:
#             continue
#         result[i] = array[non_zero_row_idx]
#         non_zero_row_idx += 1
    
#     return result

In [ ]:
# Reinsert zero rows to get the original array
# restored_array = reinsert_zero_rows(non_zero_array, zero_row_indices, array.shape)

In [ ]:
# save model
model.save('LSTM_Model.h5')
print('Model Saved!')

In [ ]:
# load model
savedModel=load_model('/content/drive/MyDrive/sharare_data/U_Net_Model.h5')
savedModel.summary()

In [ ]:
x = np.arange(y_rescaled.shape[1])  # Height indices
y = np.arange(y_rescaled.shape[0])  # Width indices

In [ ]:
# Band names
band_names = [f"band{i+1}" for i in range(7)]

# Create a dictionary to store each band as a separate variable
data_vars = {
    band_names[i]: (["y", "x"], y_rescaled[:, :, i]) for i in [0,1,2,3,4,6]
}

In [ ]:
data_vars

In [ ]:
# Create an xarray Dataset
ds = xr.Dataset(
    {
        band_names[i]: (["y", "x"], y_rescaled[:, :, i]) for i in [0,1,2,3,4,6]
    },
    coords={
        "x": x,  # Spatial x-coordinates
        "y": y,  # Spatial y-coordinates
    }
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = ds['x']
y = ds['y']

In [ ]:
bands = ["band1", "band2", "band3", "band4", "band5", "band7"]  # Replace with actual names

fig, axes = plt.subplots(3, 3, figsize=(15, 15))  # Adjust grid size as needed
axes = axes.flatten()

for i, band in enumerate(bands):
    if i < len(axes):
        data_band = ds[band]
        axes[i].pcolormesh(x, y, data_band, shading='auto', cmap='viridis')
        axes[i].set_title(band)
        axes[i].set_xlabel("X Coordinate")
        axes[i].set_ylabel("Y Coordinate")

plt.tight_layout()
plt.show()

In [ ]:
ds

In [ ]:
mirrored_ds = ds.isel(y=slice(None, None, -1))

In [ ]:
mirrored_ds = mirrored_ds.assign_coords(
    y=(-1 * mirrored_ds['y'].values) + mirrored_ds['y'].values[-1]
)

In [ ]:
x = mirrored_ds['x']
y = mirrored_ds['y']

In [ ]:
bands = ["band1", "band2", "band3", "band4", "band5", "band7"]  # Replace with actual names

fig, axes = plt.subplots(3, 3, figsize=(15, 15))  # Adjust grid size as needed
axes = axes.flatten()

for i, band in enumerate(bands):
    if i < len(axes):
        data_band = mirrored_ds[band]
        axes[i].pcolormesh(x, y, data_band, shading='auto', cmap='viridis')
        axes[i].set_title(band)
        axes[i].set_xlabel("X Coordinate")
        axes[i].set_ylabel("Y Coordinate")

plt.tight_layout()
plt.show()

In [ ]:
# Save to NetCDF file
output_file = "output_CA_LSTM_02.nc"
mirrored_ds.to_netcdf(output_file)

In [ ]:
%cd /kaggle/working

In [ ]:
from IPython.display import FileLink
FileLink(r'output_CA_LSTM_02.nc')